## Here I will take the key features from my data set and run them through weighted classificaton models, current models are Random Forest and Logit

due to class imbalance and non-linear features my models are currently correctly identifying approx 10- 50% of the failed hard drives and mis attributing approx 5 - 25% of the working drives as broken. I will implement non-linear feature treatment and svm to address this in the coming days.

In [132]:
import pandas as pd
import sklearn
import seaborn as sns

In [133]:
from sklearn.model_selection import train_test_split

In [134]:
from sklearn import svm, ensemble, linear_model, neighbors, naive_bayes

In [135]:
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix

from sklearn.metrics import precision_score, recall_score, precision_recall_curve,f1_score, fbeta_score

In [136]:
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import (RandomForestClassifier, ExtraTreesClassifier, VotingClassifier, 
                              AdaBoostClassifier, BaggingRegressor)
from sklearn import linear_model, svm, metrics, ensemble

In [137]:
from sklearn import model_selection, preprocessing

In [138]:
df = pd.read_pickle("/Users/robjohns/Data/Metis/HDfailure/Data/tot_pickle.pkl")


In [139]:
df.failure.value_counts()

0    31252
1     1486
Name: failure, dtype: int64

In [140]:
df.head()

,serial_number,failure,capacity_bytes,smart_1_normalized,smart_1_raw,smart_4_raw,smart_5_raw,smart_7_normalized,smart_9_normalized,smart_9_raw,smart_183_raw,smart_187_raw,smart_190_normalized,smart_193_raw,smart_197_raw,smart_198_raw,smart_199_raw,smart_240_raw,smart_242_raw
0,S3000FZ5,0,484095230631936,13923.0,1.387917e+10,605.0,0.0,10076.0,11470.0,601973.0,0.0,0.0,8642.0,524663.0,0.0,0.0,0.0,1.675017e+16,1.888155e+12
1,S3000QAP,0,484095230631936,13942.0,1.549529e+10,783.0,0.0,10120.0,11455.0,616584.0,0.0,0.0,8613.0,2931154.0,0.0,0.0,0.0,6.014590e+05,1.278215e+13
2,S3002HH0,0,484095230631936,13860.0,1.321871e+10,1230.0,0.0,9608.0,11891.0,239349.0,0.0,0.0,9252.0,1250581.0,0.0,0.0,0.0,2.323410e+05,3.283178e+12
3,S30034E6,0,484095230631936,13937.0,1.477775e+10,605.0,0.0,10205.0,11573.0,512600.0,0.0,0.0,9125.0,441454.0,0.0,0.0,0.0,1.662708e+16,2.761480e+12
4,S3003GAB,0,484095230631936,13887.0,1.372514e+10,949.0,0.0,9858.0,11765.0,349710.0,0.0,0.0,9654.0,1314817.0,0.0,0.0,0.0,3.432260e+05,5.174248e+12


In [141]:
df.count()

serial_number           32738
failure                 32738
capacity_bytes          32738
smart_1_normalized      32738
smart_1_raw             32738
smart_4_raw             32738
smart_5_raw             32738
smart_7_normalized      32738
smart_9_normalized      32738
smart_9_raw             32738
smart_183_raw           32738
smart_187_raw           32738
smart_190_normalized    32738
smart_193_raw           32738
smart_197_raw           32738
smart_198_raw           32738
smart_199_raw           32738
smart_240_raw           32738
smart_242_raw           32738
dtype: int64

In [142]:
y,X= df['failure'],df.drop(['failure','serial_number','capacity_bytes'],axis=1)

x_train, x_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=.2, stratify=y)

In [143]:
x_test.shape

(6548, 16)

In [144]:
#scaler = preprocessing.MinMaxScaler()
scaler = StandardScaler()

x_train = scaler.fit_transform(x_train)

x_test = scaler.transform(x_test)

In [145]:
print(len(x_train), len(x_test))

26190 6548


In [146]:
#X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2,random_state=13)

#X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=.25, random_state=43)

In [147]:
def print_metrics(model):
  
    y_pred = model.predict(x_test)

    print(metrics.classification_report(y_test, y_pred))

In [165]:
rf_weights  = {
      0: 1,
      1: 200}
RandomForest_weights = RandomForestClassifier(class_weight=rf_weights,n_estimators=400)
RandomForest_weights.fit(x_train, y_train)

print_metrics(RandomForest_weights)

              precision    recall  f1-score   support

           0       0.98      1.00      0.99      6251
           1       0.87      0.54      0.66       297

    accuracy                           0.98      6548
   macro avg       0.92      0.77      0.82      6548
weighted avg       0.97      0.98      0.97      6548



In [163]:
rf_weights  = {
      0: 1,
      1: 200}
randomforest = RandomForestClassifier(class_weight=rf_weights,n_estimators=100)

In [156]:
randomforest.fit(x_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight={0: 1, 1: 200},
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, min_impurity_decrease=0.0,
                       min_impurity_split=None, min_samples_leaf=1,
                       min_samples_split=2, min_weight_fraction_leaf=0.0,
                       n_estimators=100, n_jobs=None, oob_score=False,
                       random_state=None, verbose=0, warm_start=False)

In [157]:
randomforest.score(x_train,y_train)

0.9999236349751813

In [158]:
rf_conf=confusion_matrix(y_test, randomforest.predict(x_test))
rf_conf


array([[6227,   24],
       [ 136,  161]])

In [159]:
failperc=100*(rf_conf[0])[1]/((rf_conf[0])[0]+(rf_conf[0])[1])
print('RF Incorrectly thinks', failperc,'% are going to fail')

RF Incorrectly thinks 0.3839385698288274 % are going to fail


In [160]:
caught= 100*(rf_conf[1])[1]/((rf_conf[1])[0]+(rf_conf[1])[1])
print('RF Correctly thinks', caught,'% are going to fail')

RF Correctly thinks 54.20875420875421 % are going to fail


In [ ]:
smart_1_normalized      0.26835208
smart_242_raw           0.08309117
smart_9_normalized      0.08474709
smart_1_raw             0.07567621
smart_4_raw             0.07868987
smart_193_raw           0.07548311
smart_9_raw             0.07492018
smart_7_normalized      0.06801477
smart_190_normalized    0.06455241




smart_183_raw           0.02304617
smart_187_raw           0.02362154
smart_5_raw             0.00552174
smart_197_raw           0.03245706
smart_198_raw           0.03120192
smart_199_raw           0.00316435
smart_240_raw           0.00746034


In [168]:
RandomForest_weights.feature_importances_

array([0.26835208, 0.07567621, 0.07868987, 0.00552174, 0.06801477,
       0.08474709, 0.07492018, 0.02304617, 0.02362154, 0.06455241,
       0.07548311, 0.03245706, 0.03120192, 0.00316435, 0.00746034,
       0.08309117])

In [172]:
df.count()

serial_number           32738
failure                 32738
capacity_bytes          32738
smart_1_normalized      32738
smart_1_raw             32738
smart_4_raw             32738
smart_5_raw             32738
smart_7_normalized      32738
smart_9_normalized      32738
smart_9_raw             32738
smart_183_raw           32738
smart_187_raw           32738
smart_190_normalized    32738
smart_193_raw           32738
smart_197_raw           32738
smart_198_raw           32738
smart_199_raw           32738
smart_240_raw           32738
smart_242_raw           32738
dtype: int64